# Importing libraries

In [1]:
import os
import pandas as pd

os.chdir("..")



# Functions

## Data loading and preprocessing

In [95]:
MIN_DATE = "2017-08-17 00:00:00"

def fill_to_min_date(df, min_date, default_values):
    min_date = pd.to_datetime(min_date)
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    last_date = df['timestamp'].min()

    if last_date > min_date:
        date_range = pd.date_range(start=min_date, end=last_date - pd.Timedelta(days=1))
        df_default = pd.DataFrame(date_range, columns=['timestamp'])
        for column in ["open", "high", "low", "close", "volume", "token"]:
            df_default[column] = default_values[column]
        df = pd.concat([df_default, df])
    
    df = df.sort_values(by='timestamp')
    return df

def create_global_dataset(path, min_days=150):
    df_global = pd.DataFrame()
    for file in os.listdir(path):
        if file.split(".")[1] != "csv":
            continue
        df = pd.read_csv(path + file)
        df["token"] = file.split(".")[0]
        df["timestamp"] = pd.to_datetime(df["timestamp"])
        default_values = {
            "open": 0,
            "high": 0,
            "low": 0,
            "close": 0,
            "volume": 0,
            "token": file.split(".")[0]
        }
        df = fill_to_min_date(df, MIN_DATE, default_values)
        if len(df) > min_days:
                df_global = pd.concat([df_global, df])
    df_global.to_csv(f"./data/df_global_{path.split('/')[-2]}.csv", index=False)
    return df_global


In [96]:
def compute_change(df, periods=7):
    df["change_1d"] = df.groupby("token")["close"].pct_change(periods=1).values
    df["change_7d"] = df.groupby("token")["close"].pct_change(periods=7).values
    df["change_30d"] = df.groupby("token")["close"].pct_change(periods=30).values
    df["change_90d"] = df.groupby("token")["close"].pct_change(periods=90).values
    
    df["volume_7d"] = df.groupby("token")["volume"].rolling(window=7).sum().values
    df["volume_30d"] = df.groupby("token")["volume"].rolling(window=30).sum().values
    df["volume_90d"] = df.groupby("token")["volume"].rolling(window=90).sum().values

    df["volume_change_7d"] = df.groupby("token")["volume"].pct_change(periods=7).values
    df["volume_change_30d"] = df.groupby("token")["volume"].pct_change(periods=30).values
    df["volume_change_90d"] = df.groupby("token")["volume"].pct_change(periods=90).values
    return df

In [97]:
def select_top10_asset(df):
    df = df.loc[df["timestamp"] == df["timestamp"].max()].drop_duplicates(subset=['token'], keep='first')
    df = df.sort_values(by=["volume_30d", "change_30d", "change_90d"], ascending=False)
    return df["token"].to_list()[:10]

In [114]:
def create_multiIndex(df):
    df = df[["token", "timestamp", "open", "high", "low", "close", "volume"]]
    df = df.pivot_table(columns="token", values=["open", "high", "low", "close", "volume"], index="timestamp")
    return df

# Computing global dataset

In [115]:
df_global = create_global_dataset("./data/1d/")


In [116]:
df_global = compute_change(df_global)
df_global.head(500)


,timestamp,open,high,low,close,volume,token,change_1d,change_7d,change_30d,change_90d,volume_7d,volume_30d,volume_90d,volume_change_7d,volume_change_30d,volume_change_90d
0,2017-08-17,0.0,0.0,0.0,0.0,0.0,HBAR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2017-08-18,0.0,0.0,0.0,0.0,0.0,HBAR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2017-08-19,0.0,0.0,0.0,0.0,0.0,HBAR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2017-08-20,0.0,0.0,0.0,0.0,0.0,HBAR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2017-08-21,0.0,0.0,0.0,0.0,0.0,HBAR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,2018-12-25,0.0,0.0,0.0,0.0,0.0,HBAR,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN
496,2018-12-26,0.0,0.0,0.0,0.0,0.0,HBAR,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN
497,2018-12-27,0.0,0.0,0.0,0.0,0.0,HBAR,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN
498,2018-12-28,0.0,0.0,0.0,0.0,0.0,HBAR,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN


In [117]:
min(df_global["timestamp"])

Timestamp('2017-08-17 00:00:00')

In [127]:
asset_selected = list(set(["BTC", "ETH", "BNB", "SOL", "XRP", "ADA", "AVAX", "DOT", "MATIC"] + select_top10_asset(df_global)))
df_global = df_global.loc[df_global["token"].isin(asset_selected)].reset_index(drop=True)

In [128]:
asset_selected

['BCH',
 'BTC',
 'ETH',
 'NTRN',
 'ADA',
 'MATIC',
 'RARE',
 'C98',
 'BNB',
 'SOL',
 'LQTY',
 'HFT',
 'ID',
 'DOT',
 'XRP',
 'APE',
 'HBAR',
 'GLM',
 'AVAX']

In [129]:
new_df = create_multiIndex(df_global)
new_df.head(500)

close                                                         \
token           ADA  APE AVAX  BCH     BNB      BTC  C98  DOT     ETH  GLM   
timestamp                                                                    
2017-08-17  0.00000  0.0  0.0  0.0  0.0000  4285.08  0.0  0.0  302.00  0.0   
2017-08-18  0.00000  0.0  0.0  0.0  0.0000  4108.37  0.0  0.0  293.96  0.0   
2017-08-19  0.00000  0.0  0.0  0.0  0.0000  4139.98  0.0  0.0  290.91  0.0   
2017-08-20  0.00000  0.0  0.0  0.0  0.0000  4086.29  0.0  0.0  299.10  0.0   
2017-08-21  0.00000  0.0  0.0  0.0  0.0000  4016.00  0.0  0.0  323.29  0.0   
...             ...  ...  ...  ...     ...      ...  ...  ...     ...  ...   
2018-12-25  0.04107  0.0  0.0  0.0  5.4500  3745.79  0.0  0.0  127.60  0.0   
2018-12-26  0.04060  0.0  0.0  0.0  5.6029  3777.74  0.0  0.0  129.35  0.0   
2018-12-27  0.03611  0.0  0.0  0.0  5.1791  3567.91  0.0  0.0  113.80  0.0   
2018-12-28  0.04111  0.0  0.0  0.0  5.8811  3839.26  0.0  0.0  135.30  0.0   
2018-12-29  0.04049  0.0  0.0  0.0  5.6600  3695.32  0.0  0.0  132.89  0.0   

            ... volume                                                        
token       ...    GLM HBAR  HFT   ID LQTY MATIC NTRN RARE  SOL          XRP  
timestamp   ...                                                               
2017-08-17  ...    0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0          0.0  
2017-08-18  ...    0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0          0.0  
2017-08-19  ...    0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0          0.0  
2017-08-20  ...    0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0          0.0  
2017-08-21  ...    0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0          0.0  
...         ...    ...  ...  ...  ...  ...   ...  ...  ...  ...          ...  
2018-12-25  ...    0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0  190479475.6  
2018-12-26  ...    0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0  115977475.4  
2018-12-27  ...    0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0  113118482.1  
2018-12-28  ...    0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0  133220344.3  
2018-12-29  ...    0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0  105406488.5  

[500 rows x 95 columns]

In [130]:
new_df.shape


(2325, 95)

In [131]:
new_df.columns = pd.MultiIndex.from_product([["open", "high", "low", "close", "volume"], asset_selected], names=['price', "asset"])
new_df = new_df.dropna()

start = min(new_df.index.get_level_values(0))
df_ = new_df.loc[start:]

In [123]:
len(["open", "high", "low", "close", "volume"])*len(asset_selected)

100

In [132]:
df_

price         open                                                       \
asset          BCH    BTC    ETH   NTRN    ADA     MATIC    RARE    C98   
timestamp                                                                 
2017-08-17  0.0000  0.000   0.00    0.0    0.0   4285.08  0.0000  0.000   
2017-08-18  0.0000  0.000   0.00    0.0    0.0   4108.37  0.0000  0.000   
2017-08-19  0.0000  0.000   0.00    0.0    0.0   4139.98  0.0000  0.000   
2017-08-20  0.0000  0.000   0.00    0.0    0.0   4086.29  0.0000  0.000   
2017-08-21  0.0000  0.000   0.00    0.0    0.0   4016.00  0.0000  0.000   
...            ...    ...    ...    ...    ...       ...     ...    ...   
2023-12-24  0.5919  1.702  47.68  228.9  264.5  42991.50  0.3017  8.611   
2023-12-25  0.6242  1.797  48.18  234.8  266.9  43576.13  0.2891  9.191   
2023-12-26  0.6087  1.711  44.41  229.3  297.7  42508.93  0.3082  8.819   
2023-12-27  0.6334  1.773  42.39  264.1  324.1  43428.85  0.2913  8.694   
2023-12-28  0.6245  1.716  39.81  265.8  332.9  42581.74  0.2815  8.445   

price                        ...     volume                           \
asset           BNB     SOL  ...        SOL         LQTY         HFT   
timestamp                    ...                                       
2017-08-17   302.00  0.0000  ...        0.0          0.0         0.0   
2017-08-18   293.96  0.0000  ...        0.0          0.0         0.0   
2017-08-19   290.91  0.0000  ...        0.0          0.0         0.0   
2017-08-20   299.10  0.0000  ...        0.0          0.0         0.0   
2017-08-21   323.29  0.0000  ...        0.0          0.0         0.0   
...             ...     ...  ...        ...          ...         ...   
2023-12-24  2264.05  0.2479  ...  4596832.0  247203382.0  15947912.3   
2023-12-25  2271.35  0.2530  ...  2183400.7  157809228.0  12783867.7   
2023-12-26  2230.88  0.2468  ...  3408466.3  239716949.0  29207183.9   
2023-12-27  2378.35  0.2495  ...  3171297.4  169753136.0  24177260.7   
2023-12-28  2367.19  0.2442  ...  3042527.7  196524699.0  14692432.9   

price                                                                    \
asset               ID         DOT          XRP         APE        HBAR   
timestamp                                                                 
2017-08-17         0.0         0.0          0.0         0.0         0.0   
2017-08-18         0.0         0.0          0.0         0.0         0.0   
2017-08-19         0.0         0.0          0.0         0.0         0.0   
2017-08-20         0.0         0.0          0.0         0.0         0.0   
2017-08-21         0.0         0.0          0.0         0.0         0.0   
...                ...         ...          ...         ...         ...   
2023-12-24  50390940.0   5332652.9  163945264.1  26722957.8  33730561.1   
2023-12-25  28675751.0   4330988.8  156671024.8  26195786.5  40686659.3   
2023-12-26  70437900.0   7530896.4  405110669.5  43424392.7  22809304.9   
2023-12-27  38202913.0  13366733.5  388820801.4  15804968.4  25570243.8   
2023-12-28  32800985.0   7044452.3  205293206.9  15982115.7  17985519.7   

price                                 
asset               GLM         AVAX  
timestamp                             
2017-08-17         0.00          0.0  
2017-08-18         0.00          0.0  
2017-08-19         0.00          0.0  
2017-08-20         0.00          0.0  
2017-08-21         0.00          0.0  
...                 ...          ...  
2023-12-24  12261699.03  305003397.0  
2023-12-25  11114497.92  501529244.0  
2023-12-26  12300095.38  427204990.0  
2023-12-27   8972638.90  363959180.0  
2023-12-28   8666574.97  433883179.0  

[2325 rows x 95 columns]